In [1]:
!pip install kagglehub


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import kagglehub
jessicali9530_celeba_dataset_path = kagglehub.dataset_download('jessicali9530/celeba-dataset')

100% 1.33G/1.33G [01:49<00:00, 13.1MB/s]

Extracting files...


In [19]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.utils as vutils
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision import transforms

In [6]:
os.listdir(jessicali9530_celeba_dataset_path)

['img_align_celeba',
 'list_landmarks_align_celeba.csv',
 'list_bbox_celeba.csv',
 'list_eval_partition.csv',
 'list_attr_celeba.csv']

In [7]:
DATA_ROOT = jessicali9530_celeba_dataset_path
IMG_DIR   = os.path.join(DATA_ROOT, "img_align_celeba/img_align_celeba")
CSV_PATH  = os.path.join(DATA_ROOT, "list_attr_celeba.csv")

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
BATCH_SIZE = 256
LATENT_DIM = 256
NUM_EPOCHS = 500
WARMUP_EPOCHS = 5
IMG_SIZE = 64

ATTRS = ["Eyeglasses", "Smiling", "Mustache"]
NUM_ATTRS = 3

In [ ]:
class CelebADataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
        self.df.replace(-1, 0, inplace=True)

    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        w, h = image.size
        left = (w - 178) // 2
        top = (h - 178) // 2
        image = image.crop((left, top, left + 178, top + 178))
        if self.transform:
            image = self.transform(image)
        attrs = self.df.iloc[idx][ATTRS].values.astype(np.float32)
        return image, torch.from_numpy(attrs)

transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = CelebADataset(CSV_PATH, IMG_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=4, pin_memory=True, drop_last=True)

In [ ]:
class CVAE(nn.Module):
    def __init__(self):
        super().__init__()
        c = 128
        self.enc = nn.Sequential(
            nn.Conv2d(3 + 3, c,    4, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(c,    c*2,  4, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(c*2,  c*4,  4, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(c*4,  c*8,  4, 2, 1), nn.LeakyReLU(0.2),
        )
        self.fc_mu = nn.Linear(1024*4*4, LATENT_DIM)
        self.fc_logvar = nn.Linear(1024*4*4, LATENT_DIM)
        self.fc_dec = nn.Linear(LATENT_DIM + 3, 1024*4*4)
        self.dec = nn.Sequential(
            nn.ConvTranspose2d(1024, c*4, 4, 2, 1), nn.ReLU(),
            nn.ConvTranspose2d(c*4,  c*2, 4, 2, 1), nn.ReLU(),
            nn.ConvTranspose2d(c*2,  c,   4, 2, 1), nn.ReLU(),
            nn.ConvTranspose2d(c,    3,   4, 2, 1),
            nn.Tanh()
        )

    def encode(self, x, c):
        c = c.view(-1, 3, 1, 1).repeat(1, 1, x.size(2), x.size(3))
        x = torch.cat([x, c], dim=1)
        h = self.enc(x).view(x.size(0), -1)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c):
        z = torch.cat([z, c], dim=1)
        h = self.fc_dec(z).view(-1, 1024, 4, 4)
        return self.dec(h)

    def forward(self, x, c):
        mu, logvar = self.encode(x, c)
        z = self.reparameterize(mu, logvar)
        return self.decode(z, c), mu, logvar

model = CVAE().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=6e-4, weight_decay=1e-5)

In [ ]:
class SafeWarmupCosine(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup_steps, total_steps):
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        super().__init__(optimizer)

    def get_lr(self):
        if self.last_epoch < self.warmup_steps:
            return [base_lr * (self.last_epoch + 1) / self.warmup_steps for base_lr in self.base_lrs]
        progress = (self.last_epoch - self.warmup_steps) / (self.total_steps - self.warmup_steps)
        cos = 0.5 * (1.0 + math.cos(math.pi * progress))
        return [1e-5 + (6e-4 - 1e-5) * cos for _ in self.base_lrs]

scheduler = SafeWarmupCosine(optimizer,
                             warmup_steps=WARMUP_EPOCHS * len(dataloader),
                             total_steps=NUM_EPOCHS * len(dataloader))

def loss_function(recon, x, mu, logvar, beta=4.0):
    MSE = F.mse_loss(recon, x, reduction='sum') / x.size(0)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)
    return MSE + beta * KLD, MSE, KLD

In [ ]:
model.train()
fixed_z = torch.randn(64, LATENT_DIM, device=DEVICE)

for epoch in range(1, NUM_EPOCHS + 1):
    total_loss = 0.0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch}/{NUM_EPOCHS}")

    for imgs, attrs in pbar:
        imgs, attrs = imgs.to(DEVICE), attrs.to(DEVICE)

        optimizer.zero_grad()
        recon, mu, logvar = model(imgs, attrs)
        loss, mse, kld = loss_function(recon, imgs, mu, logvar)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        pbar.set_postfix({
            'loss': f'{loss.item():.1f}',
            'mse': f'{mse.item():.1f}',
            'lr': f'{scheduler.get_last_lr()[0]:.1e}'
        })

    avg_loss = total_loss / len(dataloader)
    print(f"\n=== Epoch {epoch} | Avg Loss: {avg_loss:.2f} | LR: {scheduler.get_last_lr()[0]:.1e} ===\n")

    if epoch % 25 == 0 or epoch == 1:
        model.eval()
        with torch.no_grad():
            def gen(attr, name):
                c = torch.tensor([attr] * 64, device=DEVICE)
                gen = model.decode(fixed_z, c)
                vutils.save_image(gen, f"samples_v3/epoch{epoch}_{name}.png",
                                  nrow=8, normalize=True, value_range=(-1,1))

            gen([0,0,0], "neutral")
            gen([1,0,0], "eyeglasses_only")
            gen([0,1,0], "smiling_only")
            gen([0,0,1], "mustache_only")
            gen([1,1,1], "all_three")

        model.train()
        print(f"Samples saved samples_3attr/epoch{epoch}_*.png")

torch.save(model.state_dict(), "cvae_eyeglasses_smiling_mustache.pth")

Epoch 1/500: 100% 791/791 [00:29<00:00, 27.20it/s, loss=942.0, mse=717.8, lr=1.2e-04]  



=== Epoch 1 | Avg Loss: 1723.62 | LR: 1.2e-04 ===

Samples saved samples_3attr/epoch1_*.png


Epoch 2/500: 100% 791/791 [00:29<00:00, 27.07it/s, loss=812.3, mse=580.1, lr=2.4e-04]



=== Epoch 2 | Avg Loss: 848.25 | LR: 2.4e-04 ===



Epoch 3/500: 100% 791/791 [00:28<00:00, 27.64it/s, loss=788.1, mse=542.2, lr=3.6e-04]


...

Epoch 390/500: 100% 791/791 [00:28<00:00, 27.82it/s, loss=602.4, mse=359.5, lr=7.9e-05]



=== Epoch 390 | Avg Loss: 608.38 | LR: 7.9e-05 ===



Epoch 391/500: 100% 791/791 [00:27<00:00, 28.42it/s, loss=609.5, mse=370.3, lr=7.8e-05]



=== Epoch 391 | Avg Loss: 608.34 | LR: 7.8e-05 ===



Epoch 392/500:  13% 101/791 [00:04<00:25, 26.69it/s, loss=595.4, mse=354.9, lr=7.8e-05]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

